In [1]:
!apt-get update
!apt-get install ffmpeg libsm6 libxext6  -y
!pip install seaborn

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1).
libxext6 is already the newest version (2:1.3.4-0ubuntu1).
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
import numpy as np
import pandas as pd
import shutil
import time
import cv2 as cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
import seaborn as sns
sns.set_style('darkgrid')
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
model = tf.keras.models.load_model('./'+'EfficientnetB7.hdf5')

2022-07-03 18:23:31.276565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23680 MB memory:  -> device: 0, name: Tesla P40, pci bus id: 0000:83:00.0, compute capability: 6.1


In [4]:
from sklearn.metrics import roc_auc_score, roc_curve, auc , accuracy_score

In [5]:
img_generator = keras.preprocessing.image_dataset_from_directory(
    './test/', 
    batch_size=5, 
    image_size=(600,600),
    shuffle=False,
)

file_paths = img_generator.file_paths

Found 225 files belonging to 1 classes.


In [6]:
test_df = pd.read_csv("./sample_solution.csv")
test_df

,ID_img,class
0,34020749806_42065966214_42113475048_2,0
1,80128313599_98196458454_79029076007_8,0
2,17820331238_48919943775_53688855463_7,0
3,70492442702_21083599816_22777758696_0,0
4,94790217016_17108156014_60668676818_2,0
...,...,...
220,60879177998_15763718934_82574532042_2,0
221,11758169966_65799840524_72283028069_1,0
222,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,0


In [7]:
predicts = []
pred_numpy = model.predict(img_generator)

for class_obj in pred_numpy:
    index, max_value = max(enumerate(class_obj), key=lambda i_v: i_v[1])
    predicts.append(index)

2022-07-03 18:23:48.877571: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


In [8]:
prediction_df=pd.DataFrame(file_paths,columns=['file'])
prediction_df['class_pred']=predicts
prediction_df['file']=prediction_df['file'].str.split('/').str[3].str.split('.').str[0]


In [9]:
rez=test_df.merge(prediction_df, left_on='ID_img', right_on='file')
rez

,ID_img,class,file,class_pred
0,34020749806_42065966214_42113475048_2,0,34020749806_42065966214_42113475048_2,2
1,80128313599_98196458454_79029076007_8,0,80128313599_98196458454_79029076007_8,1
2,17820331238_48919943775_53688855463_7,0,17820331238_48919943775_53688855463_7,2
3,70492442702_21083599816_22777758696_0,0,70492442702_21083599816_22777758696_0,2
4,94790217016_17108156014_60668676818_2,0,94790217016_17108156014_60668676818_2,1
...,...,...,...,...
220,60879177998_15763718934_82574532042_2,0,60879177998_15763718934_82574532042_2,2
221,11758169966_65799840524_72283028069_1,0,11758169966_65799840524_72283028069_1,1
222,9259096884_2251720133_44072689872_8,0,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,0,37732252922_9265441355_19052721018_3,1


In [12]:
rez=rez[['ID_img','class_pred']]
rez.columns=[['ID_img','class']]
rez

,ID_img,class
0,34020749806_42065966214_42113475048_2,2
1,80128313599_98196458454_79029076007_8,1
2,17820331238_48919943775_53688855463_7,2
3,70492442702_21083599816_22777758696_0,2
4,94790217016_17108156014_60668676818_2,1
...,...,...
220,60879177998_15763718934_82574532042_2,2
221,11758169966_65799840524_72283028069_1,1
222,9259096884_2251720133_44072689872_8,0
223,37732252922_9265441355_19052721018_3,1


In [14]:
rez.to_csv("submit3.csv", index=False)